#### MP based asynchronous testing

In [139]:
import multiprocessing as mp
import TestPool_Unit
from shutil import copyfile
import numpy as np

In [140]:
def Evaluate(result_arr):
    print('Files Processed: ', len(result_arr))
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    fully_Correct_l = 0
    fully_Correct_w = 0
    for entry in result_arr:
        (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
        recalls.append(lemma_match/n_dcsWords)
        recalls_of_word.append(word_match/n_dcsWords)

        precisions.append(lemma_match/n_output_nodes)
        precisions_of_words.append(word_match/n_output_nodes)
        if lemma_match == n_dcsWords:
            fully_Correct_l += 1
        if word_match == n_dcsWords:
            fully_Correct_w += 1
    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    print('Fully Correct Lemmawise: {}'.format(fully_Correct_l/len(recalls_of_word)))
    print('Fully Correct Wordwise: {}'.format(fully_Correct_w/len(recalls_of_word)))

In [153]:
modelFile = 'outputs/train_nnet_t987990463772.p' #margin_f
# Backup the model file
copyfile(modelFile, modelFile + '.bk')

# Create Queue, Result array
queue = mp.Queue()
result_arr = []

# Start 6 workers - 8 slows down the pc
proc_count = 6
procs = [None]*proc_count
for i in range(proc_count):
    vpid = i
    procs[i] = mp.Process(target = TestPool_Unit.pooled_Test, args = (modelFile, vpid, queue, '../NewData/skt_dcs_DS.bz2_10K/', 700))
# Start Processes
for i in range(proc_count):
    procs[i].start()

In [160]:
# Fetch partial results
while not queue.empty():
    result_arr.append(queue.get())
# Evaluate results till now
Evaluate(result_arr)

Files Processed:  4047
Avg. Micro Recall of Lemmas: 0.8446487567957679
Avg. Micro Recall of Words: 0.710321057508644
Avg. Micro Precision of Lemmas: 0.7913957420002643
Avg. Micro Precision of Words: 0.6667704683602231
Fully Correct Lemmawise: 0.3889300716580183
Fully Correct Wordwise: 0.15616506053867063


In [154]:
# Check status
for i in range(proc_count):
    p = procs[i]
    print('Process with\t vpid: {}\t ->\t pid: {}\t ->\t running status: {}'.format(i, p.pid, p.is_alive()))

Process with	 vpid: 0	 ->	 pid: 11484	 ->	 running status: True
Process with	 vpid: 1	 ->	 pid: 7020	 ->	 running status: True
Process with	 vpid: 2	 ->	 pid: 10056	 ->	 running status: True
Process with	 vpid: 3	 ->	 pid: 4140	 ->	 running status: True
Process with	 vpid: 4	 ->	 pid: 15096	 ->	 running status: True
Process with	 vpid: 5	 ->	 pid: 6564	 ->	 running status: True


In [152]:
# Properly Join
for i in range(proc_count):
    procs[i].join()

In [151]:
# Force Terminate
for p in procs:
    p.terminate()

In [865]:
np.mean([a[2] for a in result_arr])

8.3328244274809169